#### This script is created to generate trip purpose by activity purpose using CHTS data
#### To identify primary trip purpose the purpose hieracy was applied
#### Delete a nan recode of sampno==1441893 in survey_place and survey_activity and correct the plano and tripno before running the script

##### Jingyue Zhang, Last modified 1/17/2018

In [1]:
#Import libraries
from pandas import *
import numpy as np
from collections import Counter
import csv
import matplotlib.pyplot as plt
from savReaderWriter import * 

In [8]:
'''Read the data from text file and convert in into a dataframe'''
myAct=read_csv('C:/Users/jiz13007/Documents/CAMMSE/California House Travel Survey/caltrans_full_survey/caltrans_full_survey/survey_activity.csv')
print (myAct.shape)
    
# Sort the trip file
myAct.sort_values(['sampno','perno','plano','tripno','actno'], ascending=[True,True,True,True,True], inplace=True)
myAct.reset_index(drop=True,inplace = True)

#Column (25) contains character Ã
#print myAct.head(10)

myPlace=read_csv('C:/Users/jiz13007/Documents/CAMMSE/California House Travel Survey/caltrans_full_survey/caltrans_full_survey/survey_place.csv')
#print (myPlace.shape)

# Sort the place file
myPlace.sort_values(['sampno','perno','plano','tripno'], ascending=[True,True,True,True], inplace=True)
myPlace.reset_index(drop=True,inplace = True)
print myPlace.shape
#print myPlace.head()

(604710, 32)
(460523, 54)


In [9]:
'''Step1: Check if id is correct'''
#replace 'NAN' to '0' in tripno
print myAct['tripno'].isnull().sum()
myAct['tripno'].replace(np.nan,0,inplace=True)
print sum(myAct['tripno']==0)
print myPlace['tripno'].isnull().sum()
myPlace['tripno'].replace(np.nan,0,inplace=True)
print sum(myPlace['tripno']==0)

171069
171069
108778
108778


In [19]:
'''Step1.1:
Check if plano is correct
sum of consecutive n numbers 1...n = n * (n+1) /2'''
check_sampno=[]
def consecutive_plano(group):#groupby sampno and perno
    global check_sampno
    sampno=group.sampno.tolist()[0]# the entire group should have the same value
    plano=set(group.plano.tolist()) # get the unique values of plano
    maximum=max(plano)
    if sum(plano)==maximum*(maximum+1)/2:
        pass
    else:
        check_sampno.append(sampno)
    
    return group

result=myPlace.groupby(['sampno','perno']).apply(consecutive_plano)
print check_sampno

In [19]:
'''Step1.2:
Check if tripno is correct
sum of consecutive n numbers 1...n = n * (n+1) /2'''
check_sampno=[]
def consecutive_plano(group):#groupby sampno and perno
    global check_sampno
    sampno=group.sampno.tolist()[0]# the entire group should have the same value
    tripno=set(group.tripno.tolist()) # get the unique values of plano
    maximum=max(tripno)
    if sum(tripno)==maximum*(maximum+1)/2:
        pass
    else:
        check_sampno.append(sampno)
    
    return group

result=myPlace.groupby(['sampno','perno']).apply(consecutive_plano)
print check_sampno

[]


In [11]:
'''Step1.2:'''
#myAct is large so split the data into 10 portions
#groupby hh
hh_group=myAct.groupby('sampno')['perno'].max()
hh_group=DataFrame(hh_group)
hh_group.reset_index(inplace = True)
hh_group.sort_values(['sampno'], ascending=[True], inplace=True)
hh_group.reset_index(drop=True,inplace = True)
print hh_group.shape
#print hh_group.head()
hh_group['hh_indx']=hh_group.index
#print hh_group.head()
del hh_group['perno']
myAct_temp=merge(myAct, hh_group, on=['sampno'], how='left')
print myAct_temp.shape
print myAct_temp['hh_indx'].max()
Act1=myAct_temp.loc[(myAct_temp['hh_indx']>=0) &(myAct_temp['hh_indx']<=5000)]
print Act1.shape
Act2=myAct_temp.loc[(myAct_temp['hh_indx']>=5001) &(myAct_temp['hh_indx']<=10000)]
print Act2.shape
Act3=myAct_temp.loc[(myAct_temp['hh_indx']>=10001) &(myAct_temp['hh_indx']<=15000)]
print Act3.shape
Act4=myAct_temp.loc[(myAct_temp['hh_indx']>=15001) &(myAct_temp['hh_indx']<=20000)]
print Act4.shape
Act5=myAct_temp.loc[(myAct_temp['hh_indx']>=20001) &(myAct_temp['hh_indx']<=25000)]
print Act5.shape
Act6=myAct_temp.loc[(myAct_temp['hh_indx']>=25001) &(myAct_temp['hh_indx']<=30000)]
print Act6.shape
Act7=myAct_temp.loc[(myAct_temp['hh_indx']>=30001) &(myAct_temp['hh_indx']<=35000)]
print Act7.shape
Act8=myAct_temp.loc[(myAct_temp['hh_indx']>=35001) &(myAct_temp['hh_indx']<=42431)]
print Act8.shape
sum_total=len(Act1)+len(Act2)+len(Act3)+len(Act4)+len(Act5)+len(Act6)+len(Act7)+len(Act8)
print sum_total
print myAct.shape

(42431, 2)
(604710, 33)
42430
(74132, 33)
(68082, 33)
(76542, 33)
(63746, 33)
(72939, 33)
(81967, 33)
(60887, 33)
(106415, 33)
604710
(604710, 32)


In [12]:
'''Step1.2:
Check if actno is correct
sum of consecutive n numbers 1...n = n * (n+1) /2'''
check_sampno1=[]
def consecutive_actno(group):#groupby sampno and perno plano
    global check_sampno
    sampno=group.sampno.tolist()[0]# the entire group should have the same value
    actno=set(group.actno.tolist()) # get the unique values
    maximum=max(actno)
    if sum(actno)==maximum*(maximum+1)/2:
        pass
    else:
        check_sampno1.append(sampno)
    
    return group

result=Act1.groupby(['sampno','perno','plano']).apply(consecutive_actno)
print check_sampno1

[1120424, 1123111, 1123111, 1123507, 1123808, 1134470, 1135572, 1146383, 1147651, 1148923, 1164413, 1164413, 1164413, 1165342, 1165342, 1166899, 1170481, 1171450, 1197012, 1263899, 1362996, 1364491, 1365106, 1371740, 1371740, 1380695, 1381572]


In [17]:
#### actno is not consecutive, but the number will not be used and only the tripno matters, so the problem was not corrected
del myAct_temp, Act1, Act2, Act3, Act4, Act5, Act6, Act7, Act8, hh_group

#### Recode Activity Purpose Mapping
#### -9-Home:1-8;
#### 1-School:17-20;
#### 2-Primary work:9-16; 
#### 3-Work-related: 25; 
#### 4-Meal:23,31
#### 5-Shopping:27,28
#### 6-Personal Business: 24,26,29,30,32,33,34,35
#### 7-Social: 36,37
#### 8-Escorting: 22
#### 9-Other: 38, 39, 999
#### 10-Change mode: 21

In [18]:
'''Step3'''
'''recode act_pur'''
'''aggregate act_pur into 10 categories'''
def trpPur(val):
    if ((val>=1)&(val<=8)):
        return -9 # Home
    elif ((val>=17)&(val<=20)):
        return 1 #School
    elif ((val>=9)&(val<=16)):
        return 2 # Primary Work
    elif (val==25):
        return 3 #Other work
    elif ((val==23) | (val==31)):
        return 4 #Meal
    elif ((val==27) | (val==28)):
        return 5 #Shopping
    elif ((val==24) | (val==26) | (val==29) | (val==30)|(val==32) | (val==33) | (val==34) | (val==35)):
        return 6 #Personal business
    elif ((val==36) | (val==37)):
        return 7 #Social
    elif ((val==22)):
        return 8 #Escorting
    elif ((val==21)):
        return 10 #Change mode
    else:
        return 9  #Other

'''aggregate d_purpose_recode into 11 categories'''        
Temp=myAct.purpose.map(trpPur)
myAct.loc[:,'act_pur']=Temp 
print 'aggregated activity_purpose_recode categories:'
print myAct['act_pur'].value_counts().sort_index()
print myAct['purpose'].value_counts().sort_index()
del Temp

aggregated activity_purpose_recode categories:
-9     332039
 1      15635
 2      38328
 3       9940
 4      23049
 5      37190
 6      51273
 7      29311
 8      25265
 9      18607
 10     24073
Name: act_pur, dtype: int64
1     203178
2      63538
3       3461
4       4211
5       6099
6       4157
7      19989
8      27406
9      30530
10       268
11      5099
12       106
13       343
14       330
15       575
16      1077
17     11270
18      2339
19       790
20      1236
21     24073
22     25265
23      4723
24      1275
25      9940
26      5067
27     34344
28      2846
29      9204
30      7086
31     18326
32      6016
33      7376
34     10338
35      4911
36      7331
37     21980
38      4635
39     13778
99       194
Name: purpose, dtype: int64


In [38]:
'''Step3.1'''
# Identify primary activity for each trip using activity priority, the smallest value has highest priority
agg_pur=myAct.groupby(['sampno','perno','plano','tripno'])['act_pur'].min() #groupby and identify the purpose with highest priority
print agg_pur.shape #460520
agg_pur=DataFrame(agg_pur)
agg_pur.reset_index(inplace=True)
agg_pur.columns=['sampno','perno','plano','tripno','trip_pur']
print agg_pur.head()
#Merge primary purpose to myAct
myAct1=merge(myAct,agg_pur,on=['sampno','perno','plano','tripno'], how='left')
print myAct1.shape
# Identify origin purpose of the first trip (tripno==0)
myAct_ori=myAct1.loc[myAct['tripno']==0]
print myAct_ori.shape
agg_pur_ori=myAct_ori.groupby(['sampno','perno'])['trip_pur'].min()
agg_pur_ori=DataFrame(agg_pur_ori)
agg_pur_ori.reset_index(inplace=True)
agg_pur_ori.columns=['sampno','perno','first_o_pur']
print agg_pur_ori.head()

(460520L,)
    sampno  perno  plano  tripno  trip_pur
0  1031985      1      1     0.0        -9
1  1031985      1      2     1.0         7
2  1031985      1      3     2.0        -9
3  1031985      2      1     0.0        -9
4  1031985      2      2     1.0         6
(604710, 34)
(171069, 34)
    sampno  perno  first_o_pur
0  1031985      1           -9
1  1031985      2           -9
2  1032036      1           -9
3  1032036      2           -9
4  1032036      3           -9


### Identify trip purpose

In [47]:
'''Step4: Merge primary activity purpose to place file'''
#Sort the place file
myPlace.sort_values(['sampno','perno','plano','tripno'], ascending=[True,True,True,True], inplace=True)
myPlace.reset_index(drop=True,inplace = True)
print agg_pur.shape
print myPlace.shape
myPlace1=merge(myPlace, agg_pur, on=['sampno','perno','plano','tripno'], how='left')
print myPlace1.shape
print myPlace1.loc[myPlace1['trip_pur'].isnull(),'sampno']
# identified three nan values. The persons (sampno==2652524, perno==4, tripno=2,3 and sampno==2818486. perno==4, tripno==3) reported trips but not report activities
# therefore their records were deleted from place file
myPlace1=myPlace1.dropna(subset = ['trip_pur'])
print myPlace1.shape
print myPlace1['trip_pur'].isnull().sum()

(460520, 5)
(460523, 54)
(460523, 55)
321032    2652524
321033    2652524
346152    2818486
Name: sampno, dtype: int64
(460520, 55)
0


In [48]:
'''Step4.1: merge origin purpose of the first trip to place file'''
#Sort the place file
myPlace1.sort_values(['sampno','perno','plano','tripno'], ascending=[True,True,True,True], inplace=True)
myPlace1.reset_index(drop=True,inplace = True)
myPlace2=merge(myPlace1, agg_pur_ori, on=['sampno','perno'], how='left')
print myPlace2.shape
print myPlace2.loc[myPlace2['first_o_pur'].isnull(),'sampno']

(460520, 56)
Series([], Name: sampno, dtype: int64)


### Identify trip departure time and arrivial time

In [52]:

print myAct1.loc[(myAct1['sampno']==2527948)&(myAct1['perno']==1),('plano','arr_time','dep_time')]

        plano  arr_time  dep_time
369708      1   3:00:00  11:30:00
369709      1  11:31:00  13:00:00
369710      1  13:01:00  17:20:00
369711      2  17:35:00  17:40:00
369712      3  17:47:00  17:50:00
369713      3  17:51:00  18:03:00
369714      4  18:07:00  18:20:00
369715      5  18:30:00  18:25:00
369716      6  18:57:00  18:50:00
369717      7  18:57:00  19:50:00
369718      7  19:51:00   2:59:00


In [57]:
'''Step5'''
'''actno is not sorted chronologically, therefore, the trip arrive time is identified as the earliest activity start time
trip departure time is identified as the latest activity end time'''
'''Step5.1: recode time as hour*60+minutes'''
print myAct1[['arr_time','dep_time']].head()
arr_time=myAct1['arr_time'].str.split(':',expand=True).astype(int)
dep_time=myAct1['dep_time'].str.split(':',expand=True).astype(int)
print arr_time.head()
print dep_time.head()
arr_time.columns=['strt_hour','strt_min','strt_second']
dep_time.columns=['end_hour','end_min','end_second']
myAct2=concat([myAct1, arr_time], axis=1)
myAct3=concat([myAct2, dep_time], axis=1)
print myAct3.shape
print myAct3[['arr_time','strt_hour','strt_min','strt_second','dep_time','end_hour','end_min','end_second']].head()

   arr_time  dep_time
0   3:00:00   7:00:00
1   7:00:00   8:10:00
2   8:10:00   8:40:00
3   9:02:00  12:48:00
4  13:13:00  13:38:00
    0   1  2
0   3   0  0
1   7   0  0
2   8  10  0
3   9   2  0
4  13  13  0
    0   1  2
0   7   0  0
1   8  10  0
2   8  40  0
3  12  48  0
4  13  38  0
(604710, 40)
   arr_time  strt_hour  strt_min  strt_second  dep_time  end_hour  end_min  \
0   3:00:00          3         0            0   7:00:00         7        0   
1   7:00:00          7         0            0   8:10:00         8       10   
2   8:10:00          8        10            0   8:40:00         8       40   
3   9:02:00          9         2            0  12:48:00        12       48   
4  13:13:00         13        13            0  13:38:00        13       38   

   end_second  
0           0  
1           0  
2           0  
3           0  
4           0  


In [59]:
myAct3['strt_hour'].replace(0,24,inplace=True) #0 is 12am of the next day
myAct3['strt_hour'].replace(1,25,inplace=True) #1 is 1am of the next day
myAct3['strt_hour'].replace(2,26,inplace=True) #2 is 2am of the next day
myAct3['end_hour'].replace(0,24,inplace=True) #0 is 12am of the next day
myAct3['end_hour'].replace(1,25,inplace=True) #1 is 1am of the next day
myAct3['end_hour'].replace(2,26,inplace=True) #2 is 2am of the next day
print myAct3['strt_hour'].min()
print myAct3['strt_hour'].max()
print myAct3['end_hour'].min()
print myAct3['end_hour'].max()
print myAct3['strt_min'].min()
print myAct3['strt_min'].max()
print myAct3['end_min'].min()
print myAct3['end_min'].max()
print myAct3['strt_second'].min()
print myAct3['strt_second'].max()
print myAct3['end_second'].min()
print myAct3['end_second'].max()
##create strt_time and end_time
myAct3['strt_time']=(myAct3['strt_hour']-3)*60+(myAct3['strt_min'])
myAct3['end_time']=(myAct3['end_hour']-3)*60+(myAct3['end_min'])
print myAct3['strt_time'].min()
print myAct3['strt_time'].max()
print myAct3['end_time'].min()
print myAct3['end_time'].max()
print myAct3[['arr_time','strt_time','dep_time','end_time']].head()

3
26
3
26
0
59
0
59
0
0
0
0
0
1438
1
1439
   arr_time  strt_time  dep_time  end_time
0   3:00:00          0   7:00:00       240
1   7:00:00        240   8:10:00       310
2   8:10:00        310   8:40:00       340
3   9:02:00        362  12:48:00       588
4  13:13:00        613  13:38:00       638


In [64]:
'''Step5.1: check if time is correct'''
'''Check if end time is earlier than start time'''
myAct3['check_time']=myAct3['end_time']-myAct3['strt_time']
print myAct3.loc[myAct3['check_time']<0,('sampno','perno')]
# errors were identified (sampno=2527948, perno==1,plano==5,6) end_time and strt_time should be switched
myAct3.loc[(myAct3['sampno']==2527948)&(myAct3['perno']==1)&(myAct3['plano']==5),'strt_time']=925
myAct3.loc[(myAct3['sampno']==2527948)&(myAct3['perno']==1)&(myAct3['plano']==5),'end_time']=930
myAct3.loc[(myAct3['sampno']==2527948)&(myAct3['perno']==1)&(myAct3['plano']==6),'strt_time']=950
myAct3.loc[(myAct3['sampno']==2527948)&(myAct3['perno']==1)&(myAct3['plano']==6),'end_time']=957

         sampno  perno
369715  2527948      1
369716  2527948      1


In [75]:
'''Step5.2 take the ealiest strt_time and latest end_time of each place'''
strt_time_place=myAct3.groupby(['sampno','perno','plano'])['strt_time'].min()
strt_time_place=DataFrame(strt_time_place)
strt_time_place.reset_index(inplace=True)
print strt_time_place.shape
end_time_place=myAct3.groupby(['sampno','perno','plano'])['end_time'].max()
end_time_place=DataFrame(end_time_place)
end_time_place.reset_index(inplace=True)
print end_time_place.shape
print strt_time_place.head()
print end_time_place.head()

(460520, 4)
(460520, 4)
    sampno  perno  plano  strt_time
0  1031985      1      1          0
1  1031985      1      2        362
2  1031985      1      3        613
3  1031985      2      1          0
4  1031985      2      2        386
    sampno  perno  plano  end_time
0  1031985      1      1       340
1  1031985      1      2       588
2  1031985      1      3       852
3  1031985      2      1       376
4  1031985      2      2       463


In [77]:
'''Step5.3 merge strt_time_place and end_time_place to place file'''
myPlace3=merge(myPlace2,strt_time_place, on=['sampno','perno','plano'], how='left')
myPlace4=merge(myPlace3,end_time_place, on=['sampno','perno','plano'], how='left')
print myPlace2.shape
print myPlace4.shape

(460520, 56)
(460520, 58)


In [80]:
'''Step5.4 create a column to store the end time of first group of activities (tripno==0)'''
myAct3_fir=myAct3.loc[myAct3['tripno']==0]
print myAct3_fir.shape
print myAct3_fir['tripno'].max()
print myAct3_fir['plano'].max()
end_time_first=myAct3_fir.groupby(['sampno','perno'])['end_time'].max()
end_time_first=DataFrame(end_time_first)
end_time_first.reset_index(inplace=True)
end_time_first.columns=['sampno','perno','end_time_first']
print end_time_first.head()

(171069, 43)
0.0
1
    sampno  perno  end_time_first
0  1031985      1             340
1  1031985      2             376
2  1032036      1             315
3  1032036      2             332
4  1032036      3             490


In [81]:
'''Step5.4 create a column to store the end time of first group of activities (tripno==0)'''
myPlace5=merge(myPlace4,end_time_first, on=['sampno','perno'], how='left')
print myPlace5.shape

(460520, 59)


## Create trip file

In [89]:
'''Step6 Create trip file by deleting the records with no trip (tripno==0)'''
'''Step6.1: first delete the persons who did not make any trip'''
#For person who did not make any trip a record with plano==1 is still reported, need to delete these records
myPlace5['noplace']=myPlace5.groupby(['sampno','perno'])['plano'].transform('max')
myPlace5['notrip']=myPlace5.groupby(['sampno','perno'])['tripno'].transform('max')
print sum((myPlace5['noplace']==1)&(myPlace5['notrip']!=0))
print sum((myPlace5['notrip']==0)&(myPlace5['noplace']!=1))
'''Step6.2: then delete the records before the first trip (plano==1 or tripno==0)'''
myTrip=myPlace5.loc[(myPlace5['notrip']!=0)&(myPlace5['tripno']!=0)]
print myTrip.shape
print myTrip.head()

0
0
(351742, 61)
    sampno  perno  plano  vehno  tripno place_name travel_date  arr_time  \
1  1031985      1      2   97.0     1.0   REDACTED    5/1/2012   9:00:00   
2  1031985      1      3   97.0     2.0   REDACTED    5/1/2012  13:00:00   
4  1031985      2      2    1.0     1.0   REDACTED    5/1/2012   9:00:00   
5  1031985      2      3    1.0     2.0   REDACTED    5/1/2012  10:00:00   
7  1032036      1      2    1.0     1.0   REDACTED   5/12/2012   8:00:00   

   dep_time  mode   ...    per4  per5      geom  trip_pur  first_o_pur  \
1  12:00:00   6.0   ...     NaN   NaN  REDACTED       7.0           -9   
2   2:00:00   6.0   ...     NaN   NaN  REDACTED      -9.0           -9   
4  10:00:00   5.0   ...     NaN   NaN  REDACTED       6.0           -9   
5   2:00:00   5.0   ...     NaN   NaN  REDACTED      -9.0           -9   
7  15:00:00   5.0   ...     NaN   NaN  REDACTED       4.0           -9   

   strt_time end_time  end_time_first  noplace  notrip  
1        362      588   

In [90]:
# check if nan values
print myTrip['trip_pur'].isnull().sum()
print myTrip['first_o_pur'].isnull().sum()
print myTrip['strt_time'].isnull().sum()
print myTrip['end_time'].isnull().sum()
print myTrip['end_time_first'].isnull().sum()

0
0
0
0
0


In [98]:
#Rename trip_pur as d_pur
myTrip.rename(columns={'trip_pur': 'd_pur','strt_time':'act_strt_time','end_time':'act_end_time'}, inplace=True)
print myTrip[['d_pur','act_strt_time','act_end_time']].head()

   d_pur  act_strt_time  act_end_time
1    7.0            362           588
2   -9.0            613           852
4    6.0            386           463
5   -9.0            476           792
7    4.0            335           750


### Create trip origin purpose

In [92]:
# Recode trip origin purpose
def oriPur(group): #groupby smapno, perno
    tripid=group.tripno.tolist()
    sze=len(tripid)
    dpur_ls = group.d_pur.tolist()[:-1]  # Taking all but the last one
    opur_first=group.first_o_pur.tolist()[0] #Take the first one
    opur_ls=[-999 for x in xrange(sze)]
    if sze==1:
        opur_ls[0]=opur_first
    else: 
        opur_ls[0]=opur_first
        for i in xrange(len(dpur_ls)):
            dpur=dpur_ls[i]
            opur_ls[i+1]=dpur
    
    group['o_pur']=opur_ls
    return group

In [93]:
result=myTrip.groupby(['sampno','perno']).apply(oriPur)

In [96]:
print result[['sampno','perno','plano','tripno','o_pur','d_pur','first_o_pur']].head()
print result['o_pur'].value_counts()
print result['o_pur'].isnull().sum()
myTrip['o_pur']=result['o_pur']
print myTrip['o_pur'].value_counts()
print result['o_pur'].value_counts()
del result

    sampno  perno  plano  tripno  o_pur  d_pur  first_o_pur
1  1031985      1      2     1.0   -9.0    7.0           -9
2  1031985      1      3     2.0    7.0   -9.0           -9
4  1031985      2      2     1.0   -9.0    6.0           -9
5  1031985      2      3     2.0    6.0   -9.0           -9
7  1032036      1      2     1.0   -9.0    4.0           -9
-9.0     115261
 6.0      47485
 5.0      36159
 2.0      29001
 8.0      24208
 10.0     23242
 4.0      22219
 7.0      21428
 9.0      12147
 1.0      11328
 3.0       9264
Name: o_pur, dtype: int64
0
-9.0     115261
 6.0      47485
 5.0      36159
 2.0      29001
 8.0      24208
 10.0     23242
 4.0      22219
 7.0      21428
 9.0      12147
 1.0      11328
 3.0       9264
Name: o_pur, dtype: int64
-9.0     115261
 6.0      47485
 5.0      36159
 2.0      29001
 8.0      24208
 10.0     23242
 4.0      22219
 7.0      21428
 9.0      12147
 1.0      11328
 3.0       9264
Name: o_pur, dtype: int64


C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Create trip depature and arrival time

In [99]:
#start time is trip arrivial time
myTrip['trip_arr_time']=myTrip['act_strt_time']

C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [102]:
#start time is trip arrivial time
#end time is departure time of next trip
def tripTime(group): #groupby smapno, perno
    tripid=group.tripno.tolist()
    sze=len(tripid)
    act_end_time_ls=group.act_end_time.tolist()[:-1] #Take all but the last one
    act_end_first=group.end_time_first.tolist()[0] #Take the first one
    trp_dep_ls=[-999 for x in xrange(sze)]
    if sze==1: #only made one trip
        trp_dep_ls[0]=act_end_first
    else: 
        trp_dep_ls[0]=act_end_first
        for i in xrange(len(act_end_time_ls)):
            end_time=act_end_time_ls[i]
            trp_dep_ls[i+1]=end_time
    
    group['trip_dep_time']=trp_dep_ls
    return group

In [103]:
result=myTrip.groupby(['sampno','perno']).apply(tripTime)
print result[['sampno','perno','plano','tripno','act_strt_time','act_end_time','trip_dep_time','trip_arr_time']].head()
print result['trip_dep_time'].isnull().sum()
myTrip['trip_dep_time']=result['trip_dep_time']
print myTrip['trip_dep_time'].isnull().sum()
#del result

    sampno  perno  plano  tripno  act_strt_time  act_end_time  trip_dep_time  \
1  1031985      1      2     1.0            362           588            340   
2  1031985      1      3     2.0            613           852            588   
4  1031985      2      2     1.0            386           463            376   
5  1031985      2      3     2.0            476           792            463   
7  1032036      1      2     1.0            335           750            315   

   trip_arr_time  
1            362  
2            613  
4            386  
5            476  
7            335  
0
0


C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Calculate activity dwell time and trip duration time

In [108]:
'''Step7'''
myTrip['act_end_time'].replace(1439,1440, inplace=True) #should be 1440 not 1439 since the survey ended before 3:00 am of the next day
myTrip['act_dwell']=myTrip['act_end_time']-myTrip['act_strt_time']
print myTrip['act_dwell'].min()
print myTrip['act_dwell'].max()
myTrip['trip_duration']=myTrip['trip_arr_time']-myTrip['trip_dep_time']
print myTrip['trip_duration'].min()
print myTrip['trip_duration'].max()

0
1425
0
1080


C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\pandas\core\generic.py:3443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [113]:
'''Step7.1: check if sum of trip duration and act dwell is 1440'''
myTrip.loc[myTrip['tripno']==1,'initial_home']=myTrip['end_time_first']
myTrip.loc[myTrip['tripno']!=1,'initial_home']=0
myTrip.loc[(myTrip['tripno']==myTrip['notrip']),'final_home']=1440-myTrip['act_end_time']
myTrip.loc[(myTrip['tripno']!=myTrip['notrip']),'final_home']=0
print myTrip[['sampno','perno','tripno','act_strt_time','act_end_time','end_time_first','initial_home','final_home']].head(10)

C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\pandas\core\indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


     sampno  perno  tripno  act_strt_time  act_end_time  end_time_first  \
1   1031985      1     1.0            362           588             340   
2   1031985      1     2.0            613           852             340   
4   1031985      2     1.0            386           463             376   
5   1031985      2     2.0            476           792             376   
7   1032036      1     1.0            335           750             315   
8   1032036      1     2.0            770           810             315   
9   1032036      1     3.0            835           880             315   
10  1032036      1     4.0            895           940             315   
11  1032036      1     5.0            960          1440             315   
13  1032036      2     1.0            350           360             332   

    initial_home  final_home  
1          340.0         0.0  
2            0.0       588.0  
4          376.0         0.0  
5            0.0       648.0  
7          315.0   

In [118]:
check_sampno
def checkDur(group): #groupby sampno and perno
    global check_sampno
    sampno=group.sampno.tolist()[0]#take sampno
    act_dwell=sum(group.act_dwell.tolist())
    trp_dur=sum(group.trip_duration.tolist())
    initial_home=sum(group.initial_home.tolist())
    final_home=sum(group.final_home.tolist())
    check=act_dwell+trp_dur+initial_home+final_home
    if check!=1440:
        check_sampno.append(sampno)
    return group

myTrip.groupby(['sampno','perno']).apply(checkDur)
print check_sampno

[]


In [119]:
# write myTrip myAct3 and myPlace5 to pickles
myAct3.to_pickle('C:/Users/jiz13007/Documents/CAMMSE/California House Travel Survey/code/pickle data new/myAct3.pkl')
myPlace5.to_pickle('C:/Users/jiz13007/Documents/CAMMSE/California House Travel Survey/code/pickle data new/myPlace5.pkl')
myTrip.to_pickle('C:/Users/jiz13007/Documents/CAMMSE/California House Travel Survey/code/pickle data new/myTrip_process.pkl')

### Create personid, tripid by concatenating sampno+perno+tripno

In [167]:
myTrip['tripno_x1']=myTrip['tripno'].astype('float') #first convert tripno as float
myTrip['tripno_x2']=myTrip['tripno_x1'].astype('int') #then convert float to int
myTrip['tripno_x']=myTrip['tripno_x2'].apply(lambda x: '{0:0>2}'.format(x))
print myTrip['tripno_x'].head()
print myTrip['tripno_x'].value_counts()
print myTrip['tripno'].value_counts()
myTrip['plano_x']=myTrip['plano'].apply(lambda x: '{0:0>2}'.format(x))
print myTrip['plano_x'].head()
print myTrip['plano_x'].value_counts()
print myTrip['plano'].value_counts()
myTrip['perno_x']=myTrip['perno'].apply(lambda x: '{0:0>2}'.format(x))
print myTrip['perno_x'].head()
print myTrip['perno_x'].value_counts()
print myTrip['perno'].value_counts()
myTrip['trpid'] = myTrip["sampno"].map(str) + myTrip["perno_x"].map(str)+myTrip["tripno_x"].map(str)
myTrip['perid'] = myTrip["sampno"].map(str) + myTrip["perno_x"].map(str)
myTrip['plaid'] = myTrip["sampno"].map(str) + myTrip["perno_x"].map(str)+myTrip["plano_x"].map(str)

C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

0    01
1    02
2    01
3    02
4    01
Name: tripno_x, dtype: object
01    82745
02    81034
03    53409
04    43444
05    28844
06    21044
07    13563
08     9486
09     6173
10     4162
11     2663
12     1781
13     1132
14      775
15      472
16      319
17      225
18      157
19       94
20       56
21       41
22       27
23       21
24       19
25       16
26       11
27        8
28        6
29        5
32        3
31        3
30        3
33        1
Name: tripno_x, dtype: int64
1.0     82745
2.0     81034
3.0     53409
4.0     43444
5.0     28844
6.0     21044
7.0     13563
8.0      9486
9.0      6173
10.0     4162
11.0     2663
12.0     1781
13.0     1132
14.0      775
15.0      472
16.0      319
17.0      225
18.0      157
19.0       94
20.0       56
21.0       41
22.0       27
23.0       21
24.0       19
25.0       16
26.0       11
27.0        8
28.0        6
29.0        5
32.0        3
30.0        3
31.0        3
33.0        1
Name: tripno, dtype: int64


C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0    02
1    03
2    02
3    03
4    02
Name: plano_x, dtype: object
02    82745
03    81034
04    53409
05    43444
06    28844
07    21044
08    13563
09     9486
10     6173
11     4162
12     2663
13     1781
14     1132
15      775
16      472
17      319
18      225
19      157
20       94
21       56
22       41
23       27
24       21
25       19
26       16
27       11
28        8
29        6
30        5
33        3
32        3
31        3
34        1
Name: plano_x, dtype: int64
2     82745
3     81034
4     53409
5     43444
6     28844
7     21044
8     13563
9      9486
10     6173
11     4162
12     2663
13     1781
14     1132
15      775
16      472
17      319
18      225
19      157
20       94
21       56
22       41
23       27
24       21
25       19
26       16
27       11
28        8
29        6
30        5
33        3
31        3
32        3
34        1
Name: plano, dtype: int64


C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0    01
1    01
2    02
3    02
4    01
Name: perno_x, dtype: object
01    158960
02    102435
03     45859
04     28220
05     10999
06      3758
07      1130
08       381
Name: perno_x, dtype: int64
1    158960
2    102435
3     45859
4     28220
5     10999
6      3758
7      1130
8       381
Name: perno, dtype: int64


C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [168]:
print myTrip[['sampno','perno','perid','plano','plaid','tripno','trpid']].head()

    sampno  perno      perid  plano        plaid  tripno        trpid
0  1031985      1  103198501      2  10319850102     1.0  10319850101
1  1031985      1  103198501      3  10319850103     2.0  10319850102
2  1031985      2  103198502      2  10319850202     1.0  10319850201
3  1031985      2  103198502      3  10319850203     2.0  10319850202
4  1032036      1  103203601      2  10320360102     1.0  10320360101


In [169]:
del myTrip['tripno_x1']
del myTrip['tripno_x2']
del myTrip['tripno_x']

In [134]:
myTrip.sort_values(['sampno','perno','tripno'], ascending=[True,True,True], inplace=True)
myTrip.reset_index(drop=True,inplace = True)
myTrip.to_pickle('C:/Users/jiz13007/Documents/CAMMSE/California House Travel Survey/code/pickle data new/myTrip_process1.pkl')

C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


### Create hh_member variables to indicate the hh members that in the trip

In [135]:
#read hh file
hh=read_csv('C:/Users/jiz13007/Documents/CAMMSE/California House Travel Survey/caltrans_full_survey/caltrans_full_survey/survey_households.csv')
print hh.head()
print hh.shape
#sort trip file by sampno perno plano and tripno
hh.sort_values(['sampno'], ascending=[True], inplace=True)
hh.reset_index(drop=True,inplace = True)

    sampno     assn recruite_date  dow home_block_id  home_tract_id  \
0  2973777  12866.0    2013-01-25  1.0      REDACTED   6.053001e+09   
1  2973814  12866.0    2013-01-08  1.0      REDACTED   6.053001e+09   
2  2973902  13119.0    2013-02-13  6.0      REDACTED   6.053011e+09   
3  2974182  12773.0    2012-11-02  6.0      REDACTED   6.053001e+09   
4  2976556  12855.0    2013-01-08  4.0      REDACTED   6.053015e+09   

   home_county_id  home_state_id home_primarycity home_city    ...    noveh2  \
0            53.0            6.0          SALINAS   SALINAS    ...       NaN   
1            53.0            6.0          SALINAS   SALINAS    ...       NaN   
2            53.0            6.0          SALINAS   SALINAS    ...       NaN   
3            53.0            6.0          SALINAS   SALINAS    ...       NaN   
4            53.0            6.0          SALINAS   SALINAS    ...       NaN   

   noveh3 noveh4 noveh5  noveh6 noveh7 noveh8     hhwgt    exphhwgt      geom  
0     NaN   

C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [136]:
hh['persons_count'].value_counts()

2.0    16314
1.0     9139
3.0     6819
4.0     6044
5.0     2618
6.0     1000
7.0      302
8.0      185
Name: persons_count, dtype: int64

In [138]:
#Check if there are duplicates
print sum(myTrip['per1']==myTrip['per2'])
print sum(myTrip['per1']==myTrip['per3'])
print sum(myTrip['per1']==myTrip['per4'])
print sum(myTrip['per1']==myTrip['per5'])
print sum(myTrip['per2']==myTrip['per3'])
print sum(myTrip['per2']==myTrip['per4'])
print sum(myTrip['per2']==myTrip['per5'])
print sum(myTrip['per3']==myTrip['per4'])
print sum(myTrip['per3']==myTrip['per5'])
print sum(myTrip['per4']==myTrip['per5'])
print sum(myTrip['perno']==myTrip['per1'])
print sum(myTrip['perno']==myTrip['per2'])
print sum(myTrip['perno']==myTrip['per3'])
print sum(myTrip['perno']==myTrip['per4'])
print sum(myTrip['perno']==myTrip['per5'])


0
0
0
0
0
0
0
0
0
0
3
4
1
0
0


In [139]:
hhmem1=((myTrip['perno']==1)|(myTrip['per1']==1)|(myTrip['per2']==1)|(myTrip['per3']==1)|(myTrip['per4']==1)|(myTrip['per5']==1))
print hhmem1.value_counts()
hhmem2=((myTrip['perno']==2)|(myTrip['per1']==2)|(myTrip['per2']==2)|(myTrip['per3']==2)|(myTrip['per4']==2)|(myTrip['per5']==2))
print hhmem2.value_counts()
hhmem3=((myTrip['perno']==3)|(myTrip['per1']==3)|(myTrip['per2']==3)|(myTrip['per3']==3)|(myTrip['per4']==3)|(myTrip['per5']==3))
print hhmem3.value_counts()
hhmem4=((myTrip['perno']==4)|(myTrip['per1']==4)|(myTrip['per2']==4)|(myTrip['per3']==4)|(myTrip['per4']==4)|(myTrip['per5']==4))
print hhmem4.value_counts()
hhmem5=((myTrip['perno']==5)|(myTrip['per1']==5)|(myTrip['per2']==5)|(myTrip['per3']==5)|(myTrip['per4']==5)|(myTrip['per5']==5))
print hhmem5.value_counts()
hhmem6=((myTrip['perno']==6)|(myTrip['per1']==6)|(myTrip['per2']==6)|(myTrip['per3']==6)|(myTrip['per4']==6)|(myTrip['per5']==6))
print hhmem6.value_counts()
hhmem7=((myTrip['perno']==7)|(myTrip['per1']==7)|(myTrip['per2']==7)|(myTrip['per3']==7)|(myTrip['per4']==7)|(myTrip['per5']==7))
print hhmem7.value_counts()
hhmem8=((myTrip['perno']==8)|(myTrip['per1']==8)|(myTrip['per2']==8)|(myTrip['per3']==8)|(myTrip['per4']==8)|(myTrip['per5']==8))
print hhmem8.value_counts()

True     219754
False    131988
dtype: int64
False    194304
True     157438
dtype: int64
False    264186
True      87556
dtype: int64
False    286648
True      65094
dtype: int64
False    322078
True      29664
dtype: int64
False    340415
True      11327
dtype: int64
False    348438
True       3304
dtype: int64
False    350748
True        994
dtype: int64


In [140]:
myTrip['hhmember1']=0
myTrip.loc[hhmem1, 'hhmember1']=1
print myTrip['hhmember1'].value_counts()

myTrip['hhmember2']=0
myTrip.loc[hhmem2, 'hhmember2']=1
print myTrip['hhmember2'].value_counts()

myTrip['hhmember3']=0
myTrip.loc[hhmem3, 'hhmember3']=1
print myTrip['hhmember3'].value_counts()

myTrip['hhmember4']=0
myTrip.loc[hhmem4, 'hhmember4']=1
print myTrip['hhmember4'].value_counts()

myTrip['hhmember5']=0
myTrip.loc[hhmem5, 'hhmember5']=1
print myTrip['hhmember5'].value_counts()

myTrip['hhmember6']=0
myTrip.loc[hhmem6, 'hhmember6']=1
print myTrip['hhmember6'].value_counts()

myTrip['hhmember7']=0
myTrip.loc[hhmem7, 'hhmember7']=1
print myTrip['hhmember7'].value_counts()

myTrip['hhmember8']=0
myTrip.loc[hhmem8, 'hhmember8']=1
print myTrip['hhmember8'].value_counts()


C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1    219754
0    131988
Name: hhmember1, dtype: int64
0    194304
1    157438
Name: hhmember2, dtype: int64


C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0    264186
1     87556
Name: hhmember3, dtype: int64
0    286648
1     65094
Name: hhmember4, dtype: int64


C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0    322078
1     29664
Name: hhmember5, dtype: int64
0    340415
1     11327
Name: hhmember6, dtype: int64


C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0    348438
1      3304
Name: hhmember7, dtype: int64
0    350748
1       994
Name: hhmember8, dtype: int64


In [142]:
#Calculate travelers_hh
myTrip['travelers_hh_cal']=myTrip['hhmember1']+myTrip['hhmember2']+myTrip['hhmember3']+myTrip['hhmember4']+myTrip['hhmember5']+myTrip['hhmember6']+myTrip['hhmember7']+myTrip['hhmember8']

C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [159]:
#missing values in tottr, hhmem, non_hh_members
#replace missing in non_hh_members as 0, replace -1 as 0
#replace missing in hhmem as calcuated value
#replace missing in tottr as calcuated value
print myTrip['tottr'].isnull().sum()
print myTrip['hhmem'].isnull().sum()
print myTrip['non_hh_members'].isnull().sum()
myTrip['non_hh_members_recode']=myTrip['non_hh_members']
myTrip['non_hh_members_recode'].replace(np.nan,0,inplace=True)
myTrip['non_hh_members_recode'].replace(-1,0,inplace=True)
myTrip['hhmem_recode']=myTrip['hhmem']+1
myTrip.loc[myTrip['hhmem_recode'].isnull(),'hhmem_recode']=myTrip['travelers_hh_cal']
myTrip['tottr_recode']=myTrip['tottr']
myTrip.loc[myTrip['tottr_recode'].isnull(),'tottr_recode']=myTrip['travelers_hh_cal']
print myTrip['tottr_recode'].isnull().sum()
print myTrip['hhmem_recode'].isnull().sum()
print myTrip['non_hh_members_recode'].isnull().sum()

65275
65275
65275


C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0
0
0


C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [162]:
print sum(myTrip['travelers_hh_cal']!=myTrip['hhmem_recode']) #25 records do not match. Okay because only 5 hh can be reported
print sum(myTrip['tottr_recode']!=myTrip['hhmem_recode']+myTrip['non_hh_members_recode']) #8 records do not match
myTrip['tottr_cal']=myTrip['hhmem_recode']+myTrip['non_hh_members_recode']
print myTrip['tottr_cal'].min()
print myTrip['tottr_cal'].max()
print myTrip['tottr_cal'].isnull().sum()

25
8
1.0
49.0
0


C:\Users\jiz13007\AppData\Local\Continuum\Anaconda64bit\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


### Create origin/destination location variables

In [2]:
'''Read the data from text file and convert in into a dataframe'''
myAct3=read_pickle('C:/Users/jiz13007/Documents/CAMMSE/California House Travel Survey/code/pickle data new/myAct3.pkl')
print (myAct3.shape)
    
# Sort the trip file
myAct3.sort_values(['sampno','perno','plano','tripno','actno'], ascending=[True,True,True,True,True], inplace=True)
myAct3.reset_index(drop=True,inplace = True)

#Column (25) contains character Ã
#print myAct.head(10)

myPlace5=read_pickle('C:/Users/jiz13007/Documents/CAMMSE/California House Travel Survey/code/pickle data new/myPlace5.pkl')
#print (myPlace.shape)

# Sort the place file
myPlace5.sort_values(['sampno','perno','plano','tripno'], ascending=[True,True,True,True], inplace=True)
myPlace5.reset_index(drop=True,inplace = True)
print myPlace5.shape
#print myPlace.head()

myTrip=read_pickle('C:/Users/jiz13007/Documents/CAMMSE/California House Travel Survey/code/pickle data new/survey_trip.pkl')
#print (myPlace.shape)

# Sort the place file
myTrip.sort_values(['sampno','perno','plano','tripno'], ascending=[True,True,True,True], inplace=True)
myTrip.reset_index(drop=True,inplace = True)
print myTrip.shape
#print myPlace.head()

(604710, 43)
(460520, 61)
(351742, 86)


In [3]:
# Get the origin location variables of first trip (tripno==0)
loc_ori=myPlace5.loc[myPlace5['tripno']==0,('sampno','perno','tract_id','county_id','state_id')]
print loc_ori.shape
print loc_ori[['sampno','perno','tract_id','county_id','state_id']].head()
print loc_ori['tract_id'].isnull().sum()
print loc_ori['tract_id'].min()
print loc_ori['tract_id'].max()
print loc_ori['county_id'].isnull().sum()
print loc_ori['county_id'].min()
print loc_ori['county_id'].max()
print loc_ori['state_id'].isnull().sum()
print loc_ori['state_id'].min()
print loc_ori['state_id'].max()
loc_ori.columns=['sampno','perno','tract_id_first','county_id_first','state_id_first']
myTrip1=merge(myTrip,loc_ori, on=['sampno','perno'], how='left')
print myTrip1.shape

(108778, 5)
     sampno  perno  tract_id  county_id  state_id
0   1031985      1  252202.0       95.0         6
3   1031985      2  252202.0       95.0         6
6   1032036      1   17030.0       73.0         6
12  1032036      2   17030.0       73.0         6
31  1032036      3   17030.0       73.0         6
0
100.0
999999.0
0
1.0
999.0
0
1
99
(351742, 89)


In [4]:
myTrip1[['sampno','perno','tripno','tract_id','county_id','state_id','tract_id_first','county_id_first','state_id_first']].head()

,sampno,perno,tripno,tract_id,county_id,state_id,tract_id_first,county_id_first,state_id_first
0,1031985,1,1.0,252108.0,95.0,6,252202.0,95.0,6
1,1031985,1,2.0,252202.0,95.0,6,252202.0,95.0,6
2,1031985,2,1.0,251902.0,95.0,6,252202.0,95.0,6
3,1031985,2,2.0,252202.0,95.0,6,252202.0,95.0,6
4,1032036,1,1.0,17051.0,73.0,6,17030.0,73.0,6


In [5]:
myTrip1['d_tract_id']=myTrip1['tract_id']
myTrip1['d_county_id']=myTrip1['county_id']
myTrip1['d_state_id']=myTrip1['state_id']

In [8]:
#destination location is the origin location of next trip
def oriTract(group): #groupby smapno, perno
    tripid=group.tripno.tolist()
    sze=len(tripid)
    dtract_ls=group.d_tract_id.tolist()[:-1] #Take all but the last one
    tract_first=group.tract_id_first.tolist()[0] #Take the first one
    
    o_tract_ls=[-999 for x in xrange(sze)]
    
    if sze==1: #only made one trip
        o_tract_ls[0]=tract_first
        
    else: 
        o_tract_ls[0]=tract_first 
        
        for i in xrange(len(dtract_ls)):
            o_tract_id=dtract_ls[i]
            o_tract_ls[i+1]=o_tract_id

    
    group['o_tract_id']=o_tract_ls
    return group
print 'DONE'

DONE


In [ ]:
result=myTrip1.groupby(['sampno','perno']).apply(oriTract)
print sum(result['o_tract_id']==-999)
print result[['sampno','perno','tripno','tract_id_first','o_tract_id','d_tract_id']].head(10)
myTrip1['o_tract_id']=result['o_tract_id']
del result

In [ ]:
#destination location is the origin location of next trip
def oriCounty(group): #groupby smapno, perno
    tripid=group.tripno.tolist()
    sze=len(tripid)
    dcounty_ls=group.d_county_id.tolist()[:-1] #Take all but the last one
    county_first=group.county_id_first.tolist()[0] #Take the first one
    
    o_county_ls=[-999 for x in xrange(sze)]
    
    if sze==1: #only made one trip
        o_county_ls[0]=county_first
        
    else: 
        o_county_ls[0]=county_first 
        
        for i in xrange(len(dcounty_ls)):
            o_county_id=dcounty_ls[i]
            o_county_ls[i+1]=o_county_id

    
    group['o_county_id']=o_county_ls
    return group
print 'DONE'

In [ ]:
result=myTrip1.groupby(['sampno','perno']).apply(oriCounty)
print sum(result['o_county_id']==-999)
#print result[['sampno','perno','tripno','county_id_first','o_county_id','d_county_id']].tail(50)
print result.loc[result['sampno']==7211944,('perno','tripno','county_id_first','o_county_id','d_county_id')]
myTrip1['o_county_id']=result['o_county_id']
del result

In [ ]:
myTrip1.to_pickle('C:/Users/jiz13007/Documents/CAMMSE/California House Travel Survey/code/pickle data new/trip_processing3.pkl')

In [ ]:
#destination location is the origin location of next trip
def oriState(group): #groupby smapno, perno
    tripid=group.tripno.tolist()
    sze=len(tripid)
    dstate_ls=group.d_state_id.tolist()[:-1] #Take all but the last one
    state_first=group.state_id_first.tolist()[0] #Take the first one
    
    o_state_ls=[-999 for x in xrange(sze)]
    
    if sze==1: #only made one trip
        o_state_ls[0]=state_first
        
    else: 
        o_state_ls[0]=state_first 
        
        for i in xrange(len(dstate_ls)):
            o_state_id=dstate_ls[i]
            o_state_ls[i+1]=o_state_id

    
    group['o_state_id']=o_state_ls
    return group
print 'DONE'

In [ ]:
result=myTrip1.groupby(['sampno','perno']).apply(oriState)
print sum(result['o_state_id']==-999)
print result[['sampno','perno','tripno','tract_id_first','o_tract_id','d_tract_id','state_id_first','o_state_id','d_state_id']].head(10)
myTrip1['o_state_id']=result['o_state_id']
del result

### Done! Write trip file

In [170]:
##Done !!! The variables need for tour formation are created.
##Write trip file
myTrip1.to_pickle('C:/Users/jiz13007/Documents/CAMMSE/California House Travel Survey/code/pickle data new/Trips/survey_trip.pkl')

In [171]:
myTrip1[['sampno','perno','plano','tripno','trpid']].head()

,sampno,perno,plano,tripno,trpid
0,1031985,1,2,1.0,10319850101
1,1031985,1,3,2.0,10319850102
2,1031985,2,2,1.0,10319850201
3,1031985,2,3,2.0,10319850202
4,1032036,1,2,1.0,10320360101


In [173]:
myTrip1['d_pur'].value_counts().sort_index()

-9.0     114509
 1.0      11332
 2.0      29005
 3.0       9279
 4.0      22230
 5.0      36161
 6.0      47539
 7.0      22108
 8.0      24203
 9.0      12113
 10.0     23263
Name: d_pur, dtype: int64